# Data Exploration with Python

## EXERCISE 1: Reading and accessing data

### Read the survey response data

The `csv` module supports reading and writing of files in comma-separated values (CSV) and similar formats. We use `DictReader` since the first row of our survey responses file is a header. This produces a list of dictionaries, one dictionary per each individual survey response. 

A _dictionary_ is a data structure in Python that can hold key-value pairs, where we can lookup values by their key (typically a string, cf. Grok module 9). 

The `pprint` command below prints the dictionary corresponding the the first response.


In [ ]:
import csv
import pprint
data = list(csv.DictReader(open('Survey INFO3406 2018s2  - Form Responses 1.csv')))
pprint.pprint(data[0])

### Let's define constants for dictionary keys

Before moving on, let's define constants for the keys of this dictionary that will make it a bit easier to use. In our case, the keys are simply the questions from our survey in Week 1.

In [ ]:
TIMESTAMP = 'Timestamp'
BACKGROUND_INDUSTRY = 'What main industry have you worked in?'
BACKGROUND_YEARS_PROFESSIONAL = 'How many years professional experience do you have?'
BACKGROUND_YEARS_PROGRAMMING = 'How many years programming experience do you have?'
BACKGROUND_SKILLS = 'What key experience do you have?'
IMPORT_DATA_MANAGEMENT = 'Data management'
IMPORT_STATISTICS = 'Statistics'
IMPORT_VISUALISATION = 'Visualisation'
IMPORT_MACHINE_LEARNING = 'Machine Learning & Data Mining'
IMPORT_SOFTWARE_ENGINEERING = 'Software Engineering'
IMPORT_COMMUNICATION = 'Communication'
GOALS_DEFINITION = 'How would you define Data Analytics in one sentence?'
GOALS_SKILLS = 'What key skills do you want to learn?'
GOALS_ROLE = 'What kind of role would you like to go into?'
GOALS_INDUSTRY = 'What industry would you like to go into?'
IMPORT_AREAS = [
    IMPORT_DATA_MANAGEMENT,
    IMPORT_STATISTICS,
    IMPORT_VISUALISATION,
    IMPORT_MACHINE_LEARNING,
    IMPORT_SOFTWARE_ENGINEERING,
    IMPORT_COMMUNICATION
]

### Accessing data values

This allows us to access cells in a row using the column name as a key. For example, the following prints the number of years professional experience for the first respondent. Note that the csv module reads all values as strings.

In [ ]:
row = data[0]
row

In [ ]:
# Row 0 corresponds to first respondent since arrays are 0-indexed
print("response:", row[BACKGROUND_YEARS_PROFESSIONAL]) # years of professional experience
print("type:", type(row[BACKGROUND_YEARS_PROFESSIONAL])) # csv 
print("response (converted):", float(row[BACKGROUND_YEARS_PROFESSIONAL])) # years of professional experience
print("type:", type(float(row[BACKGROUND_YEARS_PROFESSIONAL]))) # convert to float

### TODO: What is the third respondent's rating for communication as integer value?

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

## EXERCISE 2: Frequency distribution and mode

### Counting data

`Counter` from the `collections` module is useful for quickly calculating frequencies. 

In [ ]:
from collections import Counter

counts = Counter()
for row in data:
    counts[row[IMPORT_COMMUNICATION]] += 1

print("Distribution of communication importance ratings:")
for k, v in sorted(counts.items()):
    print('{}: {}'.format(k, v))

### TODO: Calculate distribution of background and goal industries

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

### Calculating the mode

We can also use `Counter` to calculate the mode.

In [ ]:
mode_counter = Counter()
for row in data:
    mode_counter[row[IMPORT_COMMUNICATION]] += 1
print("Communication mode:", mode_counter.most_common(1)[0][0])

We probably will need to calculate the _mode_ more than once. It hence is a good idea to define this as our a local function which we can later call with different parameters, depending on which data we would like to calculate the mode for, and without the need to repeat all its code again and again.

In Python, one defines a local function with the **def** statement, followed by the function name and a list of arguments with which we can invoke a function later.

Our own 'mode' function is introduced and used as follows:

In [ ]:
# defines a new 'mode' function
def mode(data, column_key):
    mode_counter = Counter()
    for row in data:
        mode_counter[row[column_key]] += 1
    return mode_counter.most_common(1)[0][0]

# example on how to use the 'mode' function
print("Communication mode:", mode(data, IMPORT_COMMUNICATION))

### TODO: Calculate the mode of background and goal industries

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

## EXERCISE 3: Calculating descriptive statistics

### Cleaning float data

Which columns contained ratio data? We need to convert these to numeric types. Let's define another function since we have two ratio variables. Here we replace values that can't be converted with NaN (not a number).

In [ ]:
import numpy as np
DEFAULT_VALUE = np.nan

def clean(data, column_key, convert_function, default_value):
    special_values= {'1 year' : 1.0, '2years' : 2.0, '2 years': 2.0, 'Ten' : 10, 'Half a year': 0.5, '6 months': 0.5, '6MONTHS': 0.5, '3 Months': 0.25, '1 month': 1/12}
    for row in data:
        old_value = row[column_key]
        new_value = default_value
        try:
            if old_value in special_values.keys():
                new_value = special_values[old_value]
            else:
                new_value = convert_function(old_value)
        except (ValueError, TypeError):
            print('Replacing {} with {} in column {}'.format(row[column_key], new_value, column_key))
        row[column_key] = new_value

clean(data, BACKGROUND_YEARS_PROFESSIONAL, float, DEFAULT_VALUE)
clean(data, BACKGROUND_YEARS_PROGRAMMING,  float, DEFAULT_VALUE)

### Cleaning timestamp data

We may also want to convert timestamp values.

In [ ]:
data[0][TIMESTAMP]

In [ ]:
from datetime import datetime

FMT = "%m/%d/%Y %H:%M:%S"

def str_to_time(s):
    if isinstance(s, datetime):
        return s
    return datetime.strptime(s, FMT)

clean(data, TIMESTAMP, str_to_time, DEFAULT_VALUE)

In [ ]:
data[0][TIMESTAMP]

### Statistics with `numpy`

Once the data is converted, we can calculate descriptive statistics. `numpy` includes routines for measures of centrality and dispersion. Below we calculate descriptive statistics for professional and programming experience.

Further detail: http://docs.scipy.org/doc/numpy/reference/routines.statistics.html

In [ ]:
import numpy as np
for column_key in [BACKGROUND_YEARS_PROFESSIONAL, BACKGROUND_YEARS_PROGRAMMING]:
    v = [row[column_key] for row in data] # grab values
    print(column_key.upper())
    print("* Min..Max: {}..{}".format(np.nanmin(v), np.nanmax(v)))
    print("* Range: {}".format(np.nanmax(v)-np.nanmin(v)))
    print("* Mean: {}".format(np.nanmean(v)))
    print("* Standard deviation: {}".format(np.nanstd(v)))
    print("* Median: {}".format(np.nanmedian(v)))
    q1 = np.nanpercentile(v, 25)
    print("* 25th percentile (Q1): {}".format(q1))
    q3 = np.nanpercentile(v, 75)
    print("* 75th percentile (Q3): {}".format(q3))
    iqr = q3-q1
    print("* IQR: {}".format(iqr))

### Binning and histograms

`numpy` also provides routines for binning and producing histograms from ratio data.

NOTE RuntimeWarning due to NaN values, which are then ignored in histogram.

In [ ]:
v = []
for row in data:
    v.append(row[BACKGROUND_YEARS_PROFESSIONAL])

freqs, bins = np.histogram(v, bins=7, range=(0,35)) # calculate frequencies and bin start/end
for i, freq in enumerate(freqs):
    # Note that bins[i] <= bin_values < bins[i+1]
    bin_str = '[{}..{}]'.format(int(bins[i]), int(bins[i+1]))
    print(bin_str, ':', freq)

### TODO: Calculate histogram for programming experience

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

### EXTRA Calculate histograms with bin size of 2

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

 ## EXERCISE 4: Visualisation with matplotlib

### Making a bar chart

`matplotlib` provides functionality for creating various plots. Let's start with a frequency polygon. Note the line `%matplotlib inline` in is important in Jupyter.

In [ ]:
import matplotlib.pyplot as plt
from collections import OrderedDict
IMPORT_KEYS = ['1', '2', '3', '4', '5']
def make_importance_plot(data, column_key, title):
    c = Counter(row[column_key] for row in data)
    d = OrderedDict([(k,c[k]) if k in c else (k,0) for k in IMPORT_KEYS])
    # bars are by default width 0.8, so we'll add 0.1 to the left coordinates
    xs = [i+0.1 for i,_ in enumerate(IMPORT_KEYS)]
    plt.bar(xs, d.values())
    plt.ylabel('Number of responses')
    plt.axis([0,5,0,50])
    plt.title(title)
    plt.xticks([i + 0.5 for i, _ in enumerate(IMPORT_KEYS)], IMPORT_KEYS)
    plt.show()
for a in IMPORT_AREAS:
    title = 'Importance of {}'.format(a.lower())
    make_importance_plot(data, a, title)

### TODO: Make bar charts of known and future industries

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedError

### Making a histogram

Now let's use the `histogram` from `numpy` above to create a histograms of professional and programming experience.

In [ ]:
def iter_histogram(data, column_key):
    v = [row[column_key] for row in data] # grab values
    freqs, bins = np.histogram(v, bins=7, range=(0,30))
    for i, freq in enumerate(freqs):
        yield ('[{}..{})'.format(int(bins[i]), int(bins[i+1])), freq)
        
def make_histogram_plot(data, column_key, title):
    d = OrderedDict(iter_histogram(data, column_key))
    keys = list(d.keys())
    xs = [i+0.1 for i,_ in enumerate(keys)]
    plt.bar(xs, d.values())
    plt.ylabel('Number of responses')
    plt.axis([0,7,0,35])
    plt.title(title)
    plt.xticks([i + 0.5 for i, _ in enumerate(keys)], keys)
    plt.show()
    
make_histogram_plot(data, BACKGROUND_YEARS_PROFESSIONAL, 'Professional experience')
make_histogram_plot(data, BACKGROUND_YEARS_PROGRAMMING, 'Programming experience')

### Making a scatterplot

Finally, let's make a scatterplot to compare professional and programming experience.

In [ ]:
professional_experience = [row[BACKGROUND_YEARS_PROFESSIONAL] for row in data]
programming_experience = [row[BACKGROUND_YEARS_PROGRAMMING] for row in data]
plt.scatter(professional_experience, programming_experience)
plt.title('Professional vs programming experience')
plt.xlabel('Years of professional experience')
plt.ylabel('Years of programming experience')
plt.axis([-1,25,-1,25])
plt.show()

## EXERCISE 5: Box plots and correlation

### Visualising distributions with box plots

Mean and standard deviation are not informative for skewed data. `boxplot` is is a good visualisation for viewing and comparing distributions. It also shows outliers, e.g., values greater than `Q3+1.5*IQR` or less than `Q1-1.5*IQR`.

In [ ]:
professional_experience = [v for v in professional_experience if v is not np.nan]
programming_experience = [v for v in programming_experience if v is not np.nan]
plt.boxplot([professional_experience, programming_experience], 
            vert=False, notch=True, flierprops={'marker':'.'})
plt.axis([-1,35,0,3])
plt.yticks([1,2], ['Professional', 'Programming'])
plt.xlabel('Years experience')
plt.title('Professional vs programming experience')
plt.show()

### Calculating correlation between two variables

Pearson's r is the covariance of the two variables divided by the product of their standard deviations. Spearman rho is a common nonparametric test that is used in stead of Pearson's r when 

In [ ]:
from scipy import stats
# only keep rows where both professional and programming experience are defined
prof, prog = [], []
for row in data:
    if row[BACKGROUND_YEARS_PROFESSIONAL] is np.nan:
        continue # ignore rows with no value for professional experience
    elif row[BACKGROUND_YEARS_PROGRAMMING] is np.nan:
        continue # ignore rows with no value for programming experience
    else:
        prof.append(row[BACKGROUND_YEARS_PROFESSIONAL])
        prog.append(row[BACKGROUND_YEARS_PROGRAMMING])
print("Pearson (r, p): {}".format(stats.pearsonr(prof, prog)))

print(stats.spearmanr(prof, prog))

Calculate Kendall's tau between importance ratings

In [ ]:
print('{:.2f}'.format(3.14159))
for i,k1 in enumerate(IMPORT_AREAS):
    print(k1)
    v1 = [r[k1] for r in data]
    for j in range(i+1,len(IMPORT_AREAS),1):
        k2 = IMPORT_AREAS[j]
        v2 = [r[k2] for r in data]
        tau = stats.kendalltau(v1, v2)
        print('* {:5.2f} (p={:.3f}): {}'.format(tau.correlation, tau.pvalue, k2))

## EXERCISE 6: Text data

### Simple tokenisation and word counts

Tokenisation is the process of breaking text into it's component parts, e.g., sentences, words. Below is a simple whitespace tokeniser that also removes some leading/trailing punctuation. We can use this to count the frequency of terms acros our data analytics definitions. 

In [ ]:
def tokenise(text):
    for word in text.lower().split():
        yield word.strip('.,')

def is_valid_word(w):
    if w == '':
        return False
    else:
        return True

def iter_ds_def_words(d):
    for row in d:
        for word in tokenise(row[GOALS_DEFINITION]):
            if is_valid_word(word):
                yield word

from collections import Counter
c = Counter(iter_ds_def_words(data))
print(c)

### Removing stop words

Very common function words can be removed to focus our analysis on content words.

In [ ]:
STOP_WORDS = frozenset([ # http://www.nltk.org/book/ch02.html#stopwords_index_term
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
    'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
    'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
    'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
    'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
    'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
    'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
    'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
    ])
def is_valid_word(w):
    if w == '':
        return False
    if w.lower() in STOP_WORDS:
        return False
    else:
        return True

c = Counter(iter_ds_def_words(data))
c

### Plotting term frequencies

Now we can build a simple horizontal bar chart that displays the most common terms across data analytics definitions.

In [ ]:
import operator

def iter_word_freqs(d, min_freq=4):
    c = Counter(iter_ds_def_words(d))
    for term, freq in c.items():
        if freq >= min_freq:
            yield term, freq

d = OrderedDict([(k,v) for k,v in sorted(iter_word_freqs(data), key=operator.itemgetter(1))])
ys = [i+0.5 for i,_ in enumerate(d)]
plt.barh(ys, d.values(), align='center')
plt.yticks(ys, list(d.keys()))
plt.axis([0,50,0-0.1,len(d)+0.1])
plt.show()